
# TD 3 : Test de comparaison d'une moyenne observée à une valeur de référence

Christelle Gonindard


## 1. Présentation des données

Un clinicien évalue l’intensité de la dépression sur l’échelle de Hamilton qui varie de 0 (non déprimé) à 56 (très déprimé), chez 100 patients souffrant d’un Episode Dépressif Majeur (EDM). On veut savoir si l’acupuncture améliore l’évolution de la dépression. Pour cela, 100 malades de même état dépressif (même score de Hamilton) sont répartis en 2 groupes de 50 par tirage au sort. Le groupe I a reçu uniquement le traitement pharmacologique et le groupe II a reçu le traitement pharmacologique associé à de l’acuponcture. Le jeux de données contient les variables suivantes :

- Hamilton_G1 : Les scores de Hamilton, après 8 mois de traitement pour le groupe 1
- Hamilton_G2 : Les scores de Hamilton, après 8 mois de traitement pour le groupe 2
- Age_G1 : l’âge des patients du groupe 1
- Age_G2 : l’âge des patients du groupe 2
- poids_G1 : le poids des patients du groupe 1
- poids_G2 : le poids des patients du groupe 2
- sexe_G1 : le sexe des patients du groupe 1
- sexe_G2 : le sexe des patients du groupe 2
- Urbanisation_G1 : le niveau d’urbanisation dans lequel vivent les patients du groupe 1 (A : rural, B : urbanisation modéré, C : fortement urbanisé) 
- Urbanisation_G2 : le niveau d’urbanisation dans lequel vivent les patients du groupe 2 (A : rural, B : urbanisation modéré, C : fortement urbanisé)

Les données sont contenues dans le fichier texte : `data_Depression.txt`

La lecture des données 



In [1]:

# chargement des fonctions dont vous aurez besoin (à partir de maintenant pensez à l'ajouter vous même 
# dans les TD suivants)
C.I <- function(x,alpha){
    alpha <- 1-alpha
    mu = mean(x)
    sigma <- sd(x)
    n = length(x)
    CI_inf = mu - (qt(1 - alpha/2,n-1) * sigma/sqrt(n))
    CI_sup = mu + (qt(1 - alpha/2,n-1) * sigma/sqrt(n))
    CI = c(inf = CI_inf ,sup = CI_sup) 
    return(CI)
}



## 2. Notion de *p*-value : illustration test de **permutation** bilatéral

**Objectif :**

**Comprendre les étapes communes à tous les tests statistiques**

1. Établir les hypothèses nulles ($H_0$) et alternatives ($H_1$). L'hypothèse nulle est aussi appelée hypothèse "économe" ou "parcimonieuse" <br>
Il doit n'y avoir que 2 hypothèses : **$H_0$ faux $\Rightarrow$ $H_1$ vraie**. <br>
Il y a typiquement une seule façon de respecter $H_0$ et une multitude de façons de respecter $H_1$. <br>
2. Établir une statistique observée ($S_{obs}$) qui i) utilise l'ensemble du jeu de donnée concerné par le test et ii) décrit la tendance à dévier de $H_0$ pour se rapprocher de $H_1$.
3. Établir la *distribution nulle* de $S_{obs}$ : Si l'hypothèse nulle $H_0$ est vraie, quelle est la *densité de probabilité* de $S_{obs}$ ? Autrement dit, quelles sont les valeurs de $S_{obs}$ qui sont probables ou improbables si $H_0$ est vraie ?
4. Extraire de la *distribution nulle*, la <span style="color:red"> ***p*-value** : la probabilité d'obtenir une valeur de $S_{obs}$ au moins aussi extrême que celle observée.</span>

<img src="Figures/p-value.jpg" alt="drawing" width="600"/>


**On voudrait savoir si les scores d'Hamilton observés dans le groupe 1 chez les femmes sont égaux en moyenne aux scores observés le groupe 2.**
1. $H_0$ : ils sont égaux, la différence observée est due aux aléas d'échantillonnages. Ils n'ont qu'une seule façon d'être égaux. <br>
$H_1$ : ils sont différents, et c'est la principale cause de la différence observée. Il y a une multitude de façons d'être différents.
2. Par exemple : $S_{obs} = \hat{\mu}_1 - \hat{\mu}_2$. Mais on pourrait aussi prendre la médiane, et non la moyenne, par exemple si l'on a des valeurs extrêmes dans de petits échantillons.
3. Établir la **distribution nulle** de $S_{obs}$ : **si $H_0$ est vraie**, quelles sont les valeurs de $S_{obs}$ qui sont probables et celles qui sont improbables ? <br>
Si $H_0$ est vraie, alors, la différence observée est uniquement due à l'effet d'échantillonnage, et les scores d'Hamilton observés dans les groupes 1 et 2 proviennent de la même *population*. <br>
On peut donc simuler l'effet d'échantillonnage en mélangeant les données des groupes 1 et 2 et en les redivisant en 2 groupes pour observer la différence que l'on obtient. Si l'on répète cette opération beaucoup de fois, l'ensemble des différences observées après le mélange de nos 2 groupes constitue la distribution nulle. <br>
Répéter cette opération de très nombreuses fois à la main serait fastidieux. Nous allons donc demander à l'ordinateur de le faire pour nous. Avant l'avènement de l'informatique, des outils mathématiques ont été développés en utilisant la même logique (quelle serait la distribution, ou densité de probabilité, de $S_{obs}$ si $H_0$ était vraie ?). Cependant, pour réaliser cette déduction mathématique de la distribution nulle, il a été nécessaire de réaliser des hypothèses sur les données (ex : la normalité). Ceci rajoute des conditions d'applications aux tests statistiques qu'il convient de vérifier avant de les appliquer.

In [ ]:
head(data_depression)

# Extraction des données
femme_G1 <- data_depression$Hamilton_G1[data_depression$sexe_G1=="F"]
femme_G2 <- data_depression$Hamilton_G1[data_depression$sexe_G2=="F"]

# Affichage des données extraites
femme_G1
femme_G2

In [ ]:
# Obtenir la différence moyenne
GetMeanDiff = function(sample1, sample2){
    mean(sample1) - mean(sample2)
}
Sobs = GetMeanDiff(femme_G1,femme_G2)
Sobs

In [ ]:
# Obtenir la distribution nulle de la différence moyenne
getMixedSamples = function(sample1, sample2){
    mixed_pooled_samples = sample(c(sample1, sample2))
    return(list(mixed_pooled_samples[1:length(sample1)]
              , mixed_pooled_samples[(length(sample1)+1):length(mixed_pooled_samples)]
               ))
}

Snull = vector(length = 1e4)
for(perm in 1:1e4){
    MixedSplitedSamples = getMixedSamples( femme_G1, femme_G2 )
    Snull[perm] = GetMeanDiff(MixedSplitedSamples[[1]]  , MixedSplitedSamples[[2]])
}


In [ ]:
# Afficher la distribution nulle et la comparer visuellement à la statistique observée
hist(Snull)
abline(v = Sobs, col=2, lwd=3)
abline(v = -Sobs, col=2, lwd=1.5, lty=2)

In [ ]:
# Calculer la p-value (probabilité d'obtenir une valeur de 'Sobs' au moins aussi extrême que celle observée) : 
# On peut être extrème, soit du coté négatif
Pneg = Snull < -abs(Sobs)
Pneg
Pneg = sum(Pneg) / 1e4
# soit du coté positif
Ppos = Snull >  abs(Sobs)
Ppos = sum(Ppos) / 1e4
cat("p-value = ",Pneg+Ppos," : ")

 **La probabilité d'observer une différence aussi forte que ce qu'on l'a observé si $H_0$ est vraie est de 0.7319**. Il n'y a donc pas d'évidence que la différence soit liée à autre chose qu'un effet d'échantillonnage. Le test est donc **inconclusif** : il est tout à fait possible qu'il y ait une différence, mais s'il y en a une, les données ne permettent pas de la détecter.

## 3. Test bilatéral


**Objectif :**

**On voudrait savoir si les scores d'Hamilton observés dans le groupe 1 chez les femmes sont égaux en moyenne à la valeur de référence 29.**

Nous devons réaliser un test de Student comparant la moyenne estimée par la fonction `mean()` avec la valeur de référence escomptée E = 29 dans notre cas.

*Le principe de ce test est simple* : il va vérifier si l’écart entre la moyenne observée $\mu$ dans l’échantillon et la valeur de référence $\mu _0$ est suffisamment éloigné de 0 avec un certain risque d’erreur $\alpha$ pour que les deux moyennes soient considérées comme différentes.
Le test calcule donc la différence entre $\mu$ et $\mu _0$ et divise par son écart-type pour normaliser :

$$ T = \frac{\mu - \mu _0}{\sqrt{\frac{\hat{\sigma} ^2}{n} }}=\frac{\mu - \mu _0}{\frac{\hat{\sigma}}{\sqrt{n}}}$$

<img src="Figures/Test_bilateral.jpg" />

$H_0$ : le score d'Hamilton chez les femmes du groupe 1 est égal à 29. <br>
$H_1$ : le score d'Hamilton chez les femmes du groupe 1 n’est pas égal à 29.

**Extraction de l'échantillon** :
**Taille de l'échantillon** :


**Vérification des conditions d'application**

Il faut vérifier la taille de l'échantillon : 
- Si la taille de l’échantillon est grande ($n \geq 30$) : pas de condition d’application
- Si la taille de l’échantillon est petite ($n < 30$) : il faut que les données suivent une loi normale

Vérifiez les conditions d'application 

Hypothèses du test de Shapiro : <br>
$H_0$ : ??? <br>
$H_1$ : ???

Conclusion, au seuil 5%, la *p*-value > 0.05 donc on ne peut pas rejeter $H_0$ : les données ne dévient pas significativement d'une loi normale

**Réalisation du test de Student**

In [ ]:
t.test(femme_G1,mu=29)

Conclusion, au seuil 5%, la *p*-value < 0.05 donc on rejette $H_0$, le score d'Hamilton pour les femmes du groupe 1 est différent de 29. La probabilité d'observer des données aussi improbable que les notres, si la moyenne de la population est de 29 est de $2.36\times 10^{-7}$.

## 4. Test unilatéral

**Objectif :**

On voudrait savoir si les scores d'Hamilton observés dans le groupe 2 chez les femmes sont supérieurs en moyenne à 0.
Le score d'Hamilton varie de 0 (non déprimé) à 56 (très déprimé). On sait donc par construction qu'il ne peut pas être inférieur à 0. Dans ce type de cas particulier, on peut choisir un test unilatéral.

$H_0$ : le score d'Hamilton chez les femmes du groupe 2 est égal à 29.<br>
$H_1$ : le score d'Hamilton chez les femmes du groupe 2 est supérieur à 29.<br>

<img src="Figures/Test_unilateral.jpg" />


In [ ]:
# l'argument alternative = greater permet la sélection du test unilatéral
t.test(femme_G2,mu=0,alternative="greater")

*Conclusion* : Au seuil 5%, la *p*-value < 0.05 donc on rejette l’hypothèse $H_0$ donc le score de
Hamilton pour les femmes du groupe 2 est supérieur à 0.

## 5. Exercices d'application sur le jeu de données data_depression.txt

###  Exercice 1

Répondez aux questions suivantes, les tests seront réalisés au seuil $\alpha=0.02$ :

1. Est-ce que les scores d'Hamilton observés dans le groupe 2 chez les femmes est inférieur en moyenne à 56 ?
2. Est-ce que les scores d'Hamilton observés dans le groupe 1 chez les hommes est supérieur en moyenne à 0 ?
3. Est-ce que les scores d'Hamilton observés dans le groupe 2 chez les hommes est supérieur en moyenne à 0 ?
4. Est-ce que les scores d'Hamilton observés dans le groupe 1 chez les femmes est supérieur en moyenne à 0 ?

Concluez.


### Exercice 2 (à faire à la maison pour s'entrainer)

Répondez aux questions suivantes, les tests seront réalisés au seuil $\alpha=0.01$ :

1. Est-ce que les scores d'Hamilton observés dans le groupe 2 dans les zones rurales sont différents en moyenne à 29?
2. Est-ce que les scores d'Hamilton observés dans le groupe 2 dans les zones à urbanisation modéré sont différents en moyenne à 29?
3. Est-ce que les scores d'Hamilton observés dans le groupe 2 dans les zones fortement urbanisées sont inférieurs en moyenne à 25?
4. Est-ce que les scores d'Hamilton observés dans le groupe 1 dans les zones rurales sont inférieurs en moyenne à 39?
5. Est-ce que les scores d'Hamilton observés dans le groupe 1 dans les zones à urbanisation modéré sont supérieurs en moyenne à 29?
6. Est-ce que les scores d'Hamilton observés dans le groupe 1 dans les zones fortement urbanisées sont différents en moyenne à 25?

Concluez.